In [36]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
import pandas as pd
from functions import main_cleaning
from sklearn.preprocessing import StandardScaler

pd.set_option('display.max_columns', None) 

df_raw = pd.read_csv("dielectron.csv")

In [38]:
df_cleaned = main_cleaning(df_raw)
df_cleaned.reset_index(drop=True, inplace=True)

e:\Iván\Documentos\Clase\Informatica\Beca IronHack\Temario\PROYECTOS\Proyecto_Final_IronHack\functions.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["is_same_charge"] = df["Q1"] == df["Q2"]
e:\Iván\Documentos\Clase\Informatica\Beca IronHack\Temario\PROYECTOS\Proyecto_Final_IronHack\functions.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["is_outlier"] = False


### Database creation

In [ ]:
# Event has one value repeated. We have 2 options: 
# We can either drop it or create an ID for Event. We will do the last.

df_cleaned[df_cleaned["Event"] == 418006834]

In [5]:
# "Run" table

df_run = df_cleaned[["Run"]].drop_duplicates().copy()

df_run["date_run"] = pd.NaT

df_run.rename(columns={"Run": "run_num"}, inplace=True)

In [6]:
# "Particle A" table

df_particle_a = df_cleaned[["E1", "px1", "py1", "pz1", "pt1", "eta1", "phi1", "Q1"]].copy()

df_particle_a.rename(columns={"E1": "energy", "px1": "px", "py1": "py", "pz1": "pz", "pt1": "pt", "eta1": "eta", "phi1": "phi", "Q1": "charge"}, inplace=True)

In [7]:
# "Particle B" table

df_particle_b = df_cleaned[["E2", "px2", "py2", "pz2", "pt2", "eta2", "phi2", "Q2"]].copy()

df_particle_b.rename(columns={"E2": "energy", "px2": "px", "py2": "py", "pz2": "pz", "pt2": "pt", "eta2": "eta", "phi2": "phi", "Q2": "charge"}, inplace=True)

In [4]:
# from functions import import_to_sql

# import_to_sql(df_run, "run")
# import_to_sql(df_particle_a, "particle_a")
# import_to_sql(df_particle_b, "particle_b")


In [146]:
# ID for particles

from functions import make_query

id_partA_query = "SELECT id_part FROM particle_a"
id_partA = make_query(id_partA_query)

id_partB_query = "SELECT id_part FROM particle_b"
id_partB = make_query(id_partB_query)

In [147]:
# Event table

df_event = df_cleaned[["Event", "Run", "M"]].copy()

df_event["id_partA"] = id_partA
df_event["id_partB"] = id_partB

df_event.rename(columns={"Event": "event_num", "Run": "run_num", "M": "invariant_mass"}, inplace=True)

In [148]:
# import_to_sql(df_event, "event")

# MACHINE LEARNING

### Los resultados de MSE, MAE y R2 para KNN sin normalizar eran: 
MAE: 17.717622715597013  
MSE: 12.581512831873468   
R2: 0.507506217561434  
### So I will normalize/standarize.

In [39]:
df_ml = df_cleaned.dropna()

In [6]:
features = df_ml[["Run", "px1", "py1", "pz1", "px2", "py2", "pz2", "is_same_charge", "is_outlier"]]
target = df_ml[["M"]]

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.20, random_state=42)

### Normalization

In [8]:
from sklearn.preprocessing import MinMaxScaler

normalizer = MinMaxScaler()

X_train_norm= normalizer.fit_transform(X_train)        

X_test_norm = normalizer.transform(X_test)

In [174]:
X_train_norm = pd.DataFrame(X_train_norm, columns = X_train.columns)   
X_test_norm = pd.DataFrame(X_test_norm, columns = X_test.columns)

### I will try 5 different models

In [ ]:
from sklearn.ensemble import RandomForestRegressor

from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, r2_score

# Define models
models = {
  'KNeighborsRegressor': KNeighborsRegressor(),
  'Linear Regression': LinearRegression(),
  'Decision Tree': DecisionTreeRegressor(random_state=42),
  'Random Forest': RandomForestRegressor(random_state=42, n_estimators=100),
  'SVR': SVR(),
}   

# Train and evaluate models
results = {}

for name, model in models.items():
  # Train the model
    model.fit(X_train_norm, y_train)
  
  # Make predictions
    y_pred_norm = model.predict(X_test_norm)

    mae_standardized = mean_absolute_error(y_test, y_pred_norm)
    mse_standardized = root_mean_squared_error(y_test, y_pred_norm)
    r2_standardized = r2_score(y_test, y_pred_norm)
  
  # Store results
    results[name] = {
        'mae' : mae_standardized,
        'mse' : mse_standardized,
        'R2' : r2_standardized
  }

# Print results
for name, result in results.items():
  print(f"\n{name}:")
  print(f"Mean Absolute Error: {result['mae']}")
  print(f"Root Mean Squared Error: {result['mse']}")
  print(f"R2: {result['R2']}")

# Compare R2_value
r2_values = {name: result['R2'] for name, result in results.items()}
best_model = max(r2_values, key=r2_values.get)

print("\nModel Accuracy Comparison:")
for name, accuracy in r2_values.items():
  print(f"{name}: {accuracy:.4f}")

print(f"\nBest performing model: {best_model} with accuracy {r2_values[best_model]:.4f}")

### Standarization

In [34]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)        

X_test_scaled = scaler.transform(X_test)

In [35]:
X_train_scaled = pd.DataFrame(X_train_scaled, columns = X_train.columns)   
X_test_scaled = pd.DataFrame(X_test_scaled, columns = X_test.columns)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, r2_score

# Define models
models = {
  'KNeighborsRegressor': KNeighborsRegressor(),
  'Linear Regression': LinearRegression(),
  'Decision Tree': DecisionTreeRegressor(random_state=42),
  'Random Forest': RandomForestRegressor(random_state=42, n_estimators=100),
  'SVR': SVR(),
}   

# Train and evaluate models
results = {}

for name, model in models.items():
  # Train the model
    model.fit(X_train_scaled, y_train)
  
  # Make predictions
    y_pred_scaled = model.predict(X_test_scaled)

    mae_standardized = mean_absolute_error(y_test, y_pred_scaled)
    mse_standardized = root_mean_squared_error(y_test, y_pred_scaled)
    r2_standardized = r2_score(y_test, y_pred_scaled)
  
  # Store results
    results[name] = {
        'mae' : mae_standardized,
        'mse' : mse_standardized,
        'R2' : r2_standardized
  }

# Print results
for name, result in results.items():
  print(f"\n{name}:")
  print(f"Mean Absolute Error: {result['mae']}")
  print(f"Root Mean Squared Error: {result['mse']}")
  print(f"R2: {result['R2']}")

# Compare R2_value
r2_values = {name: result['R2'] for name, result in results.items()}
best_model = max(r2_values, key=r2_values.get)

print("\nModel Accuracy Comparison:")
for name, accuracy in r2_values.items():
  print(f"{name}: {accuracy:.4f}")

print(f"\nBest performing model: {best_model} with accuracy {r2_values[best_model]:.4f}")

### We take Standarized Random Forest as the best model

### Now, we are going to try RF removing columns

In [40]:
features_lower = df_ml[["Run", "pz1", "pz2", "is_same_charge", "is_outlier"]]
target_lower = df_ml[["M"]]

In [41]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features_lower, target_lower, test_size=0.20, random_state=42)

In [42]:
scaler_lower = StandardScaler()

X_train_scaled = scaler_lower.fit_transform(X_train)        

X_test_scaled = scaler_lower.transform(X_test)

In [43]:
X_train_scaled = pd.DataFrame(X_train_scaled, columns = X_train.columns)   
X_test_scaled = pd.DataFrame(X_test_scaled, columns = X_test.columns)

In [ ]:
rf_lower = RandomForestRegressor(random_state=42, n_estimators=100)

rf_lower.fit(X_train_scaled, y_train)
  
y_pred_lower_scaled = rf_lower.predict(X_test_scaled)

mae_standardized = mean_absolute_error(y_test, y_pred_lower_scaled)
mse_standardized = root_mean_squared_error(y_test, y_pred_lower_scaled)
r2_standardized = r2_score(y_test, y_pred_lower_scaled)

mae_standardized, mse_standardized, r2_standardized

Todo -> (2.3447513779118077, 3.9098505538185626, 0.9760166321541155)

Sin py -> (5.030465963967166, 8.80652660636964, 0.8783256160020667)

Sin py ni pz -> (16.016459623359527, 21.49041157214318, 0.2754317177361222)

Sin px ni py -> (9.648483653205867, 15.70953893783305, 0.6128165981972764)

In [ ]:
features_scaled = scaler_lower.transform(features_lower)

y_pred_lower_real = rf_lower.predict(features_scaled)


mae_standardized = mean_absolute_error(target_lower, y_pred_lower_real)
mse_standardized = root_mean_squared_error(target_lower, y_pred_lower_real)
r2_standardized = r2_score(target_lower, y_pred_lower_real)

mae_standardized, mse_standardized, r2_standardized

In [ ]:
df_ml["pred"] = y_pred_lower_real

df_ml

In [ ]:
from sklearn.model_selection import cross_val_score

# Aplicamos la validación cruzada en el conjunto de entrenamiento
scores = cross_val_score(rf_lower, X_train_lower_scaled, y_train_lower, cv=5, scoring='r2')

print("Cross-validated R2 scores:", scores)
print("Mean cross-validated R2:", scores.mean())


# Trying with Random Forest and Hyperparameter Tuning

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Define hyperparameter space for Random Forest
param_grid_rf = {
    'n_estimators': [100, 300, 500],  # Número de árboles en el bosque
    'max_depth': [None, 10, 30, 50],        # Máxima profundidad de cada árbol
    #'min_samples_split': [2, 5, 10],                # Número mínimo de muestras requeridas para dividir un nodo
    'min_samples_leaf': [1, 2, 4],                  # Número mínimo de muestras requeridas para estar en una hoja
    #'max_features': ['auto', 'sqrt',],       # Número de características a considerar para la mejor división
    #'bootstrap': [True, False],                     # Método para seleccionar muestras para entrenar cada árbol
    #'criterion': ['absolute_error', 'friedman_mse', 'poisson', 'squared_error'],               # Función para medir la calidad de una división
}


# Initialize base XGBoost model
rf_regressor = RandomForestRegressor()

# Set up KFold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)

# Configure GridSearchCV for RF
grid_search_rf = GridSearchCV(rf_regressor, param_grid_rf, cv=kfold, scoring='neg_mean_squared_error')

# Train GridSearchCV with XGBoost
grid_search_rf.fit(X_train, y_train)

# Extract the optimal XGBoost model
best_rf = grid_search_rf.best_estimator_

# Predict using the optimal model
y_pred_train_best_rf = best_rf.predict(X_train)
y_pred_test_best_rf = best_rf.predict(X_test)

# Cross-validation scores for XGBoost using best_xgb
mse_scores_rf = cross_val_score(best_rf, X_train, y_train, cv=kfold, scoring=mse_scorer)
r2_scores_rf = cross_val_score(best_rf, X_train, y_train, cv=kfold, scoring=r2_scorer)
mae_scores_rf = cross_val_score(best_rf, X_train, y_train, cv=kfold, scoring=mae_scorer)

# Compile results into a DataFrame for XGBoost with Grid Search
evaluation_metrics_xgb = {
    'Model': ['RandomForest (Grid Search)'],
    'Avg_MSE_CV': [np.mean(mse_scores_rf)],
    'Std_MSE_CV': [np.std(mse_scores_rf)],
    'Avg_R2_CV': [np.mean(r2_scores_rf)],
    'Std_R2_CV': [np.std(r2_scores_rf)],
    'Avg_MAE_CV': [np.mean(mae_scores_rf)],
    'Std_MAE_CV': [np.std(mae_scores_rf)],
    'MSE_Train': [root_mean_squared_error(y_train, y_pred_train_best_rf)],
    'R2_Train': [r2_score(y_train, y_pred_train_best_rf)],
    'MAE_Train': [mean_absolute_error(y_train, y_pred_train_best_rf)],
    'MSE_Test': [root_mean_squared_error(y_test, y_pred_test_best_rf)],
    'R2_Test': [r2_score(y_test, y_pred_test_best_rf)],
    'MAE_Test': [mean_absolute_error(y_test, y_pred_test_best_rf)]
}

rf_results_df = pd.DataFrame(evaluation_metrics_xgb)

# # Append results to the existing DataFrame
# results_df = pd.concat([results_df, xgb_results_df], ignore_index=True)
# results_df

Sin px -> Mean cross-validated R2: 0.8715048380715927

Sin px ni py -> Mean cross-validated R2: 0.6095551354410731

# A new model: XGBoost

In [50]:
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
import xgboost as xgb

xg_model = xgb.XGBRegressor()

grid = {
    'n_estimators': [100, 200, 300, 400, 500],            # Número de árboles
    'max_depth': [3, 5, 7, 9, 11],                        # Profundidad máxima de los árboles
    'learning_rate': [0.01, 0.05, 0.1, 0.2, 0.3],        # Tasa de aprendizaje
    'subsample': [0.6, 0.8, 1.0],                         # Proporción de muestras utilizadas para entrenar
    'colsample_bytree': [0.6, 0.8, 1.0],                 # Proporción de características utilizadas para entrenar
    'gamma': [0, 0.1, 0.2, 0.3],                          # Reducción de pérdida requerida para hacer una partición adicional
    'reg_alpha': [0, 0.1, 1, 10],                         # Regularización L1
    'reg_lambda': [0, 0.1, 1, 10]                        # Regularización L2
}


xg_model_randomized_search = RandomizedSearchCV(estimator = xg_model, param_distributions = grid, n_iter = 10, cv = 5, n_jobs = -1) 

In [51]:
xg_model_randomized_search.fit(X_train_scaled, y_train)
  
y_pred_scaled = xg_model_randomized_search.predict(X_test_scaled)

mae_standardized = mean_absolute_error(y_test, y_pred_scaled)
mse_standardized = root_mean_squared_error(y_test, y_pred_scaled)
r2_standardized = r2_score(y_test, y_pred_scaled)

mae_standardized, mse_standardized, r2_standardized

(9.099255470977, 14.89928796896824, 0.6517261382987032)

In [52]:
from sklearn.model_selection import cross_val_score

# Aplicamos la validación cruzada en el conjunto de entrenamiento
scores = cross_val_score(xg_model_randomized_search, X_train_scaled, y_train, cv=5, scoring='r2')

print("Cross-validated R2 scores:", scores)
print("Mean cross-validated R2:", scores.mean())


Cross-validated R2 scores: [0.65769514 0.6539168  0.65243117 0.64477293 0.63621046]
Mean cross-validated R2: 0.6490053011901388
